In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, col, to_date, when, date_format, month

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("CovidStats") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
df = spark.read.csv("./complete.csv", header=True, inferSchema=True)

The Day with max covid cases

In [4]:
daily_cases = df.groupBy("Date").agg(sum("Total Confirmed cases").alias("Total_Cases"))
max_cases_day = daily_cases.orderBy(daily_cases["Total_Cases"].desc()).limit(1)
max_cases_day  = max_cases_day.withColumn("Formatted_Date", date_format(col("Date"), "dd-MMM-yyyy"))
result = max_cases_day.collect()[0]

formatted_date_str = result['Formatted_Date']
print(f"It was on {formatted_date_str} max covid cases recorded")

It was on 06-Aug-2020 max covid cases recorded


The state has the second-largest number of covid cases.

In [5]:
state_cases = df.groupBy("Name of State / UT").agg(sum("Total Confirmed cases").alias("Total_Cases"))
second_largest_state = state_cases.orderBy(state_cases["Total_Cases"].desc()).limit(2).collect()[1]

print(f"State with second largest covid cases {second_largest_state['Name of State / UT']}")

State with second largest covid cases Tamil Nadu


Which Union Territory has the least number of death.

In [6]:
states = [
    "Andaman and Nicobar Islands", "Chandigarh", "Dadra and Nagar Haveli and Daman and Diu",
    "Delhi", "Jammu and Kashmir", "Ladakh", "Lakshadweep", "Puducherry"
]

pattern = "(?i)" + "|".join(states)

states = df.filter(df["Name of State / UT"].rlike(pattern))
deaths = states.groupBy("Name of State / UT").agg(sum("Death").alias("Total_Deaths"))
state_with_low_deaths = deaths.orderBy("Total_Deaths").first()
state_with_low_deaths_non_zero = deaths.filter(col("Total_Deaths") > 0).orderBy("Total_Deaths").first()

print(f"Least deaths recorded in '{state_with_low_deaths_non_zero["Name of State / UT"]}'")

Least deaths recorded in 'Dadra and Nagar Haveli and Daman and Diu'


The state has the lowest Death to Total Confirmed cases ratio.

In [7]:
state_ratios = df.groupBy("Name of State / UT").agg(
    sum("Death").alias("Total_Deaths"),
    sum("Total Confirmed cases").alias("Total_Cases")
).withColumn(
    "Death_to_Cases_Ratio",
    when(col("Total_Cases") != 0, col("Total_Deaths") / col("Total_Cases")).otherwise(None)
)

lowest_ratio_state = state_ratios.orderBy("Death_to_Cases_Ratio").first()

print(lowest_ratio_state["Name of State / UT"])

Mizoram


Find which month the Most Newer recovered cases.

In [8]:
new_cases_by_month = df.groupBy(date_format('Date','MMMM').alias("Month")).agg(sum("New recovered").alias("Total New Recovered"))
most_recover_cases = new_cases_by_month.orderBy(new_cases_by_month["Total New Recovered"].desc()).first()["Month"]
print(f"The month with most new recoveries is '{most_recover_cases}'")

The month with most new recoveries is 'July'
